In [1]:
import pandas as pd
import re
import numpy as np

c:\Users\jtsanchez\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\jtsanchez\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\Users\jtsanchez\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df = pd.read_csv('SO_ADV_data (7).csv')
df


,TIME_MESSAGE,MESSAGE,REGION
0,2023-06-14 9:47:19 PM,Mindanao Grid-06/14/2023 21:48: WMPC DG01 cut-...,CMIN
1,2023-06-14 9:46:52 PM,Mindanao Grid-06/14/2023 21:48: WMPC DG01 cut-...,CMIN
2,2023-06-14 9:44:44 PM,Mindanao Grid-06/14/2023 21:45: WMPC DG05 cut-...,CMIN
3,2023-06-14 9:44:18 PM,Mindanao Grid-06/14/2023 21:45: WMPC DG05 cut-...,CMIN
4,2023-06-14 9:29:17 PM,Mindanao Grid-06/14/2023 21:29: WMPC DG07 cut-...,CMIN
...,...,...,...
21156,2021-12-11 1:24:00 PM,VRCC-12/11/2021 09:21: 34.5 KV CARCON - Sangi...,CVIS
21157,2021-12-11 1:24:00 PM,VRCC-12/11/2021 13:00: GRID UPDATE AS OF 1:00 ...,CVIS
21158,2021-12-11 1:23:00 PM,LRCC-12/11/2021 12:53: GRID UPDATE AS OF 1:00 ...,CLUZ
21159,2021-12-11 1:22:00 PM,LRCC-12/11/2021 09:04: Tayabas-Mauban 230kV Li...,CLUZ


### Remove duplicates

In [3]:
#remove duplicates

df.duplicated(subset=['MESSAGE'])

0        False
1         True
2        False
3         True
4        False
         ...  
21156    False
21157    False
21158    False
21159    False
21160    False
Length: 21161, dtype: bool

In [4]:
df.drop_duplicates(subset='MESSAGE', inplace=True)

In [5]:
df.duplicated(subset=['MESSAGE']).value_counts()

False    14630
Name: count, dtype: int64

### Keyword detection

In [6]:
#keyword lists
    
outage_keywords = [
"shut down", "shutdown", "out", "outage",
"desynchronize", "shut", "out", "planned outage", "trip", "tripping", "tripped",
"offline", "unavailable", "energize", "de-energize","de-energized", "emergency"
'failed to synchronized','failed to synchronize'
] 


restoration_keywords=[
'online', 'on-line' 'energize', 'restored', "sync","synchronize", 
"synchronized", 're-energized', 're energize', 're-energize', 'energization'
]


#derated_keywords=['derated'] #i others na lang to
#note: cut-in/cut-out - reserve/ RPS- classify as other



In [7]:
keyword_lists = {'restoration': restoration_keywords,'outage': outage_keywords}  
                    #i put "restoration" first coz restoration updates also contain outage keywords

def get_matched_list(message):
    for name, keywords in keyword_lists.items():
        if any(re.search(fr'\b{re.escape(keyword)}\b', message, flags=re.IGNORECASE) for keyword in keywords):
            return name
    return "other"  # if no match


df['MESSAGE TYPE'] = df['MESSAGE'].str.lower().apply(get_matched_list)


df

,TIME_MESSAGE,MESSAGE,REGION,MESSAGE TYPE
0,2023-06-14 9:47:19 PM,Mindanao Grid-06/14/2023 21:48: WMPC DG01 cut-...,CMIN,outage
2,2023-06-14 9:44:44 PM,Mindanao Grid-06/14/2023 21:45: WMPC DG05 cut-...,CMIN,outage
4,2023-06-14 9:29:17 PM,Mindanao Grid-06/14/2023 21:29: WMPC DG07 cut-...,CMIN,outage
6,2023-06-14 8:59:30 PM,Mindanao Grid-06/14/2023 21:00: Sibulan Hedcor...,CMIN,restoration
7,2023-06-14 8:55:57 PM,Mindanao Grid-06/14/2023 20:56: Sibulan Hedcor...,CMIN,outage
...,...,...,...,...
21156,2021-12-11 1:24:00 PM,VRCC-12/11/2021 09:21: 34.5 KV CARCON - Sangi...,CVIS,outage
21157,2021-12-11 1:24:00 PM,VRCC-12/11/2021 13:00: GRID UPDATE AS OF 1:00 ...,CVIS,other
21158,2021-12-11 1:23:00 PM,LRCC-12/11/2021 12:53: GRID UPDATE AS OF 1:00 ...,CLUZ,other
21159,2021-12-11 1:22:00 PM,LRCC-12/11/2021 09:04: Tayabas-Mauban 230kV Li...,CLUZ,outage


In [8]:
# then add if line/plant/transformer update (Gelo code)

line_pattern = r"(?<!\bon[-\s])\bline\b(?![-\s]on\b)" # regex to exclude "on line" and "on-line"


# Create a new column 'Category' to classify the message
df['CATEGORY'] = np.where(df['MESSAGE'].str.contains(r"\d+\s*MW"), "plant",
                                    np.where(df['MESSAGE'].str.contains(r"\d+\s*MVA"), "transformer",
                                             np.where(df['MESSAGE'].str.contains(r"\d+\s*kV"), "line",
                                                      np.where(df['MESSAGE'].str.contains(r"transformer", case=False), "transformer",
                                                               np.where(df['MESSAGE'].str.contains(line_pattern, case=False), "line", # modified to exclude "on line" and "on-line"
                                                                        np.where(df['MESSAGE'].str.contains('weather', case=False),'weather','plant'))))))

In [9]:
df.to_csv('with message type.csv', index=False)